# Assigment 6

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

1. Generate a folder with the name of your group in this folder **GitHub\Diplomado_PUCP\_output\sbs\municipal**. The code should run even if the folder exists.
2. Scrape all the available information in this [link](https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=C). Save an excel file by year and month request. The excel file should be name using the month and year of the sheet. 

In [1]:
#Instalación de librerías
#!pip install selenium
#!pip install webdriver-manager

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

#Library
# ---
# For scrape
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# Options driver
from selenium.webdriver.chrome.options import Options

# For dataframes
import pandas as pd
from pandas import json_normalize

# For simulate human behavior.
import time
from time import sleep
import random
from selenium.webdriver.common.by import By

# Clear data
import unidecode
#unaccented_string = unidecode.unidecode(accented_string)

# Manipulate Json files
import json
import re
import numpy as np

In [3]:
# Setting
options= Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)
# we open our page
# Cajas municipales
driver.get("https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=C")

C:\Users\U0468\AppData\Local\Temp\ipykernel_24284\1018851190.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)


In [4]:
# Dropdown button list
dropdown_button_year = driver.find_element( By.ID, "ctl00_cphContent_rAnio")
dropdown_button_year.click()

# List of years options
list_year = driver.find_element( By.ID, "ctl00_cphContent_rAnio_DropDown" )
years_list = list_year.find_elements( By.TAG_NAME, 'li')
list_year_options = len(years_list)

# Dropdown button list
dropdown_button_month = driver.find_element( By.ID, "ctl00_cphContent_rMes")
dropdown_button_month.click()

# # List of months options
list_month = driver.find_element( By.ID,  "ctl00_cphContent_rMes_DropDown" )
months_list = list_month.find_elements( By.TAG_NAME, 'li')
list_month_options = len(months_list)
driver.find_element( By.ID, "ctl00_cphContent_btnConsultaMensual").click()
driver.get("https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=C")

In [5]:
year_numbers_opt = np.array( [1, 0 ] + np.arange(2, list_year_options).tolist() )
month_numbers_opt = np.array( [1, 0 ] + np.arange(2, list_month_options).tolist() )

In [6]:
years_info_list = []
months_info_list = []
tables_info_list = []

for year_number in year_numbers_opt[3:10]:
    
    # Dropdown button list
    dropdown_button_year = driver.find_element( By.ID, "ctl00_cphContent_rAnio") 
    dropdown_button_year.click()

    # List of years options
    time.sleep(0.5)
    driver.find_element( By.XPATH,  f"/html/body/form/div[1]/div/ul/li[{year_number + 1 }]" ).click()
    year_text = driver.find_element( By.ID, "ctl00_cphContent_rAnio").text

    
        
    for month_number in month_numbers_opt[3:10]:
        
        # Dropdown button list
        dropdown_button_month = driver.find_element( By.ID, "ctl00_cphContent_rMes") 
        dropdown_button_month.click()

        # # List of months options
        time.sleep(0.5)
        driver.find_element( By.XPATH,  f"/html/body/form/div[1]/div/ul/li[{month_number + 1 }]" ).click()
        month_text = driver.find_element( By.ID, "ctl00_cphContent_rMes").text
        print( year_text )
        print( month_text )
        
        # Click consultar
        driver.find_element( By.ID,"ctl00_cphContent_btnConsultaMensual").click()
        time.sleep(5)
        
        year_float = int(year_text)
        
        try:
            # Switch elements
            driver.switch_to.frame(driver.find_element( By.ID,"ctl00_cphContent_pnContenidoAnteriorMnASP"))
            section_tab_table = driver.find_element( By.XPATH,  "/html/body/form/table/tbody" )
            table_html = section_tab_table.get_attribute('outerHTML')
            table_pd = pd.read_html( table_html )

            # Cleaning tablr
            table_clean_1 = table_pd[1].iloc[5:, :].copy().reset_index( drop = True )

            # Changing switch
            driver.switch_to.default_content()

            print("Table Tipo 1 - Cambiar de Frame y una tabla simple")

        except:
            print("No table Tipo 1")
            table_clean_1 = "Null"
            
            
        try:
            # Switch elements
            driver.switch_to.frame(driver.find_element( By.ID,"ctl00_cphContent_pnContenidoAnteriorMnASP"))
            section_tab_table = driver.find_element( By.XPATH,  "/html/body/form/table/tbody" )
            table_html = section_tab_table.get_attribute('outerHTML')
            table_pd = pd.read_html( table_html )

            # generate table objects
            table0, table1 = table_pd[0], table_pd[1]
            table_clean_2 = table1.copy()

            # Number of rows of second table
            tb1_rows = table1.shape[0]
            drop_rows = np.arange( 1, tb1_rows + 2 )

            # Clean first table
            table0 = table0.drop( drop_rows, axis = 0 )

            # Assign values to new table
            col_name = table0.iloc[0, 0]
            col_values = table0.iloc[1:, 0].tolist()
            table_clean_2.insert( loc = 0, column= col_name, value = col_values)
            
            print("Table Tipo 2 - Cambiar de Frame y una tabla Compleja")
            
             # Changing switch
            driver.switch_to.default_content()
            
        except:
            print("No table Tipo 2")
            table_clean_2 = "Null"
            

        try:
            # Get tables
            section_id_table = driver.find_element( By.ID, "ctl00_cphContent_rpgActualMn_OT" )
            table_html = section_id_table.get_attribute('outerHTML')
            table_pd = pd.read_html( table_html )

            # generate table objects
            table0, table1 = table_pd[0], table_pd[1]
            table_clean_3 = table1.copy()

            # Number of rows of second table
            tb1_rows = table1.shape[0]
            drop_rows = np.arange( 1, tb1_rows + 2 )

            # Clean first table
            table0 = table0.drop( drop_rows, axis = 0 )

            # Assign values to new table
            col_name = table0.iloc[0, 0]
            col_values = table0.iloc[1:, 0].tolist()
            table_clean_3.insert( loc = 0, column= col_name, value = col_values)
            
            print("Table Tipo 3 - No Cambiar de Frame y una tabla Compleja")
            
        except:
            print("No table Tipo 3")
            table_clean_3 = "Null"
            
            
        # Check what to save
        if ( ( isinstance( table_clean_3, str ) == False ) and ( table_clean_3.shape[1] > 5 ) ):
            
            # Data with no frame
            table_clean_3.to_excel( fr"C:\Users\Anzony\Documents\GitHub\Selenium_Scrappers\B_RawData\sbs/B_RawData/cajas_rurales/table_clean_{year_text}_{month_text}.xlsx", index = False )
            
            years_info_list.append( year_float)
            months_info_list.append( month_text )
            tables_info_list.append( "Table 3" )
            print("guardo3")
            
        elif ( ( isinstance( table_clean_2, str ) == False ) and ( table_clean_2.shape[1] > 5 ) ):
            
            # Data with frame and 2 tables
            table_clean_2.to_excel( fr"C:\Users\Anzony\Documents\GitHub\Selenium_Scrappers\B_RawData\sbs/B_RawData\cajas_rurales/table_clean_{year_text}_{month_text}.xlsx", index = False, header = False )
            # To break
            years_info_list.append( year_float)
            months_info_list.append( month_text )
            tables_info_list.append( "Table 2" )
            print("guardo2")
            
        elif ( ( isinstance( table_clean_1, str ) == False ) and ( table_clean_1.shape[1] > 5 ) ):
            # Data with frame and 1 table
            table_clean_1.to_excel( fr"C:\Users\Anzony\Documents\GitHub\Selenium_Scrappers\B_RawData\sbs/B_RawData\cajas_rurales/table_clean_{year_text}_{month_text}.xlsx", index = False , header = False )
            
            years_info_list.append( year_float)
            months_info_list.append( month_text )
            tables_info_list.append( "Table 1" )
            print("guardo1")
            
        # No table information
        # Go to a previous day
        else:
            # Day ago
            years_info_list.append( year_float)
            months_info_list.append( month_text )
            tables_info_list.append( "No Table" )
            
        driver.switch_to.default_content()
                

2020
Abril
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja


OSError: Cannot save file into a non-existent directory: 'C:\Users\Anzony\Documents\GitHub\Selenium_Scrappers\B_RawData\sbs\B_RawData\cajas_rurales'